In [38]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tvcs3CZf1bnzkMLjfa0wQ2mwTRLEAo8x-iW4kQ7NEB65',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'avnrdrproj-donotdelete-pr-itcxutjunubhqr'
object_key = 'kaggle (1).zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/



In [39]:
!pip install -q kaggle

In [40]:
!mkdir ~/.kaggle # creating a kaggle directory

mkdir: cannot create directory ‘/home/wsuser/.kaggle’: File exists


In [41]:
pwd

'/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset'

In [42]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [43]:
ls

AVNR_DR.h5  image-classification-model_new.tgz  kaggle/  testing/  training/


In [44]:
cd /home/wsuser/work/kaggle

/home/wsuser/work/kaggle


In [45]:
ls

 diabetic-retinopathy-level-detection.zip   kaggle.json
 inception-diabetic.h5                     'preprocessed dataset'/


In [46]:
!cp kaggle.json ~/.kaggle/  # copying json file to folder

In [47]:
!chmod 600 ~/.kaggle/kaggle.json  # changing the permissions to json

In [48]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

diabetic-retinopathy-level-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip diabetic-retinopathy-level-detection.zip

Archive:  diabetic-retinopathy-level-detection.zip
replace inception-diabetic.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd /home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset

In [ ]:
ls

In [ ]:
imageSize = [299, 299]

trainPath = r"/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset/training"

testPath = r"/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset/testing"

In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [ ]:
# don't train existing weights
for layer in xception.layers:
    layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [26]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [27]:
pwd

'/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset'

In [28]:
ls

testing/  training/


In [29]:
training_set = train_datagen.flow_from_directory(r'/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset/training',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [31]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

/tmp/wsuser/ipykernel_159/1842758803.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
3/3 [==============================] - 45s 14s/step - loss: 3.4023 - accuracy: 0.5938
Epoch 2/5
3/3 [==============================] - 39s 11s/step - loss: 4.6037 - accuracy: 0.6562
Epoch 3/5
3/3 [==============================] - 40s 12s/step - loss: 5.2705 - accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 37s 11s/step - loss: 7.9608 - accuracy: 0.6354
Epoch 5/5
3/3 [==============================] - 40s 11s/step - loss: 5.1905 - accuracy: 0.6458


In [32]:
model.save('AVNR_DR.h5')

In [33]:
!tar -zcvf image-classification-model_new.tgz AVNR_DR.h5

AVNR_DR.h5


In [34]:
ls -1

AVNR_DR.h5
image-classification-model_new.tgz
testing/
training/


In [35]:
!pip install watson-machine-learning-client --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.9/538.9 kB 30.8 MB/s eta 0:00:00


In [37]:
n3UkLrzk3VvvjjpI7I0RCQ0eBbPqHIxc-mZcOQnI-haB

NameError: name 'n3UkLrzk3VvvjjpI7I0RCQ0eBbPqHIxc' is not defined

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com",
                "apikey":"50N1KIC8lc0jdAzc9S_WhMK6JT9pi5mUySsy7-kZhy7I"}
client = APIClient(wml_credentials)

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()